In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from textblob import TextBlob
import zipfile
import os



C:\Users\ML\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# import raw dataset
csv_file = './data/lyrics.csv'

if not os.path.isfile(csv_file):
    with zipfile.ZipFile(f'{csv_file}.zip', 'r') as zip_ref:
        zip_ref.extractall('./data')
        
df = pd.read_csv(csv_file)
rows, _= df.shape
print('datasættet i sin "raw" form indeholder:')
print(f'{rows} antal rækker')


datasættet i sin "raw" form indeholder:
362237 antal rækker


### Funktioner til senere brug

In [22]:
def normalize(df, new_col_name, col_to_norm):
    '''
    ref: https://en.wikipedia.org/wiki/Normalization_(statistics)
    '''
    df = df.copy()
    max = df[col_to_norm].max()
    min = df[col_to_norm].min()

    df[new_col_name] = df[col_to_norm].apply(lambda val: (val-min)/(max-min))
    return df

def _count_words(words):
    try:
        return len(words.split())
    except:
        return 0 #TODO: better error handling, maybe not return 0

def word_count(df, new_col_name, col_with_lyrics):
    df = df.copy()
    df[new_col_name] = df[col_with_lyrics].apply(lambda words: _count_words(words))
    return df

def remove_outliers(df, col_to_process, low=.05, high=.95):
    df = df.copy()
    min, max = df[col_to_process].quantile([low,high])
    df = df[(df[col_to_process] >= min) & (df[col_to_process] <= max)]
    return df.reset_index(drop=True)

def analyze_sentiment(df):
    df = df.copy()
    res = df['lyrics'].apply(lambda txt : TextBlob(txt).sentiment)
    df['polarity'] = res.apply(lambda x: x[0])
    df['subjectivity'] = res.apply(lambda x: x[1])
    return df

def sentence_avg_word_length(df, new_col_name, col_with_lyrics):
    df[new_col_name] = df[col_with_lyrics].astype(str).apply(_sentence_avg_word_length)
    return df

def _sentence_avg_word_length(sentence):
    res = sum(len(word.split()) for word in sentence) / len(sentence.split())**3
    return res

def analyze_word_class(df):
    blobs = df['lyrics'].apply(lambda txt : TextBlob(txt).tags)
    df['nouns'] = blobs.apply(lambda word_list: _count_word_class(word_list, 'NN'))
    df['adverbs'] = blobs.apply(lambda word_list: _count_word_class(word_list, 'RB'))
    df['verbs'] = blobs.apply(lambda word_list: _count_word_class(word_list, 'VB'))
    
    return df

def _count_word_class(words, word_class):
    count = 0
    for w in words:
        if w[1] == word_class:
            count = count + 1
    return count / 100

def prepare_data(df, data_cols, label_col, training_size=1000, test_size=250):
    labels = df_cp[label_col].value_counts().keys().tolist()
    train_data, train_labels, test_data, test_labels = [], [], [], []
    
    # shuffle dataset
    df = df.copy().sample(frac=1).reset_index(drop=True)
    
    for label in labels:
        data = df[df[label_col] == label]
        # kun hvis der er nok eksempler, ift. training_size og test_size, ud fra den pågældende label
        if len(data) > training_size + test_size:
            data = data.reset_index(drop=True)
            train_data += data[data_cols][0:training_size].values.tolist()
            train_labels += data[label_col][0:training_size].values.tolist()
            test_data += data[data_cols][training_size:training_size+test_size].values.tolist()
            test_labels += data[label_col][training_size:training_size+test_size].values.tolist()
    
    # da modellen kun kan trænes med numpy arrays, så skal listerne lige konverteres
    train_data = np.asarray(train_data)
    train_labels = np.asarray(train_labels)
    test_data = np.asarray(test_data)
    test_labels = np.asarray(test_labels)
    
    return (train_data, train_labels), (test_data, test_labels)
        
        

### udvind features og tilføj til datasæt
Hvis det er første gang dette step køres vær opmærksom på, at der laves tunge sproglige analyser af sangteksterne. Det vil derfor resulterer i, siden der er MANGE rækker data, at det kan tage 40+ minutter at udvinde alle features hvis hele datasættet benyttes. Det kan derfor anbefales, at man tage en mindre sample af datasættet. 

Hvis dette step ER kørt, så burde der være gemt en `.pkl` i `./data` som gør, at feature genereringen kan springes over. 

Hvis man gerne vil generere et nyt feature, måske fordi man gerne vil have et mindre datasæt ved ændring af `sample` variablen, så lav om `FEATURE_DATASET_FILE` variablen, så det gamle feature datasæt kan beholdes. Der skal ligges mærke til, at når der udtages sample af datasættet, så ligges de enkelte udvalgte rækker tilbage. Dette er en IKKE en rigtig måde at gøre det på, da der så kan fremkomme duplikationer af rækker. Det er derimod for, at lave et "proof of concept" med mulighed for, at benytte alle genre i træning af modellen.

In [34]:
FEATURE_DATASET_FILE = './data/feature_dataset.pkl'
SAMPLE_SIZE = 1000 # <-- ændre denne hvis et anden størrelse datasæt ønskes

if not os.path.isfile(FEATURE_DATASET_FILE):
    df_cp = df.copy() # <-- kopi af importeret datasæt

    ###
    # Oprydning i raw datasæt
    ###
    # fjern rækker med nan værdier
    df_cp = df_cp.dropna()

    # fjern uønskede genre (Not Available & Other)
    df_cp = df_cp[(df_cp.genre != 'Not Available') & (df_cp.genre != 'Other')]

    ###
    # Udtaget stikprøve af datasæt
    ###
    grouped = df_cp.groupby('genre')
    grouped = grouped.apply(lambda x: x.sample(n=SAMPLE_SIZE, replace=True))
    df_cp = grouped.reset_index(drop=True)
    
    ###
    # Tilføj features til datasæt
    ###
    # tilføj kategoriske numeriske værdier for genre
    print('generate categorical values for genre...', end='')
    df_cp.genre = pd.Categorical(df_cp.genre)
    df_cp['genre_code'] = df_cp.genre.cat.codes
    print('DONE')
    
    # optæl ord i sangtekst
    print('generate number-of-words feature and normalize...', end='')
    df_cp = word_count(df_cp, 'num_words', 'lyrics')
    # normaliser antal ord i sangtekst
    df_cp = normalize(df_cp, 'num_words_nm', 'num_words')
    # fjern outliers ud fra antal ord i sangtekst
    df_cp = remove_outliers(df_cp, 'num_words')
    print('DONE')
    
    # optæl gennemsnitlig ordlængde i sangtekst
    print('generate avg-word-length feature and normalize...', end='')
    #df_cp = df_cp[df_cp.lyrics.apply(type) == str] # lyrics MUST be type string
    df_cp = sentence_avg_word_length(df_cp, 'avg_word_len', 'lyrics')
    df_cp = normalize(df_cp, 'avg_word_len_nm', 'avg_word_len')
    print('DONE')
    
    # sentiments analyse
    print('generate sentiments analyzis...', end='')
    df_cp = analyze_sentiment(df_cp)
    print('DONE')
    
    # optæl ord-klasser i sangtekst
    print('generate word-class counts...', end='')
    df_cp = analyze_word_class(df_cp)
    print('DONE')
    
    # fjern col 'index'
    df_cp.drop(['index'], axis=1, inplace=True)
    
    print('saving feature dataset to pickle...', end='')
    df_cp.to_pickle(FEATURE_DATASET_FILE)
    print('DONE')
else:
    print('reading feature dataset from pickle...', end='')
    df_cp = pd.read_pickle(FEATURE_DATASET_FILE)
    print('DONE')

max_nw, min_nw = df_cp.num_words.max(), df_cp.num_words.min()
max_awl, min_awl = df_cp.avg_word_len.max(), df_cp.avg_word_len.min()

reading feature dataset from pickle...DONE


### Klargør træning og test data/labels
Da modellen ikke skal testes på data den også trænes på, for ikke at give et misvisende billede af hvor god modellen er til, at genkende genre fra en sangtekst den ikke har set før.

Der er flere variabler der gerne må ændres på for, at se hvilket resultat det giver til modellen evne til, at genkende genren:

- `features` er en liste med de karakteristika som modellen skal lærer fra. Det er ikke altid sikker, at alle features bidrager til en bedre evne til, at genkende genren. Derfor kan der fjernes fra denne `list` variabel.
- `train_size`/`test_size` bestemmer hvor stort et datasæt modellen skal trænes/testes på. Da ikke alle genre optræder i datasættet lige mange gange vil, hvis man vælger en for stor `train_size` + `test_size`, bestemte genre ikke blive tilføjet til train/test datasættet da `(antal af sangtekste fra en bestemt genre) >= train_size + test_size`. Dette sørger `prepare_data()` for at blive realiseret.

Der vil, efter nedestående blok bliver kørt, blive vist hvilke genre modellen trænes i, at kunne genkende.

In [27]:
features = ['num_words_nm', 'avg_word_len_nm', 'subjectivity', 'polarity', 'nouns', 'adverbs', 'verbs']
output_labels = 'genre_code'
train_size = 100
test_size = 20

# klargør data til model
(train_data, train_labels), (test_data, test_labels) = prepare_data(df_cp, features, output_labels, train_size, test_size)

# Vis genre ud fra kategori kode
print('Genre modellen trænes til at genkende samt hvilken genre_code genren har:')
for code in np.unique(test_labels):
    print(code, df_cp[df_cp.genre_code == code].genre[0])


Genre modellen trænes til at genkende samt hvilken genre_code genren har:
0 Country
1 Electronic
2 Folk
3 Hip-Hop
4 Indie
5 Jazz
6 Metal
7 Pop
8 R&B
9 Rock


### Setup netværk lag

- `input_nodes` er antallet af inputs parametrer/features
- `hidden_nodes` anbefalet antal er svarende til et tal mellem input og output nodes
- `output_nodes` er antallet af "labels" kategorier man forsøger at klassificerer for

In [28]:
input_nodes = len(features)
hidden_nodes = 4
output_nodes = 12

model = keras.Sequential([
    keras.layers.Dense(input_nodes),
    keras.layers.Dense(hidden_nodes, activation=tf.nn.sigmoid),
    keras.layers.Dense(output_nodes, activation=tf.nn.sigmoid)
])


### Compile modellen
Før modellen er klar til træning, mangler den nogle flere indstillinger. Disse er tilføjet under compiling:

- Loss function — Denne måler hvor præcis modellen er under træning. Vi vil minimerer denne funktion til, at "styre" modellen i den rigtige retning.
- Optimizer — Denne afgører hvordan modellen er opdateret, baseret på det data den ser 
- Metrics — Brugt til at monitorerer under træningen og testing trin.

In [29]:
model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### Træning af modellen

In [30]:
model.fit(train_data, train_labels, epochs=40)

Epoch 1/40
1000/1000 [==============================] - 0s 236us/step - loss: 2.4652 - acc: 0.1000
Epoch 2/40
1000/1000 [==============================] - 0s 28us/step - loss: 2.4521 - acc: 0.1000
Epoch 3/40
1000/1000 [==============================] - 0s 24us/step - loss: 2.4396 - acc: 0.1000
Epoch 4/40
1000/1000 [==============================] - 0s 25us/step - loss: 2.4276 - acc: 0.1000
Epoch 5/40
1000/1000 [==============================] - 0s 24us/step - loss: 2.4160 - acc: 0.1000
Epoch 6/40
1000/1000 [==============================] - 0s 25us/step - loss: 2.4050 - acc: 0.1000
Epoch 7/40
1000/1000 [==============================] - 0s 25us/step - loss: 2.3944 - acc: 0.1000
Epoch 8/40
1000/1000 [==============================] - 0s 27us/step - loss: 2.3846 - acc: 0.1000
Epoch 9/40
1000/1000 [==============================] - 0s 25us/step - loss: 2.3751 - acc: 0.1000
Epoch 10/40
1000/1000 [==============================] - 0s 25us/step - loss: 2.3663 - acc: 0.1000
Epoch 11/40
1000/1

### Evaluerer præcisionen

In [31]:
test_loss, test_acc = model.evaluate(test_data, test_labels)

print('Test accuracy:', test_acc)

200/200 [==============================] - 0s 244us/step
Test accuracy: 0.105


### Predictions

In [49]:
lyric = '''
input din sangtekst her!!
'''


dic = {'lyrics': [lyric]}
predict_df = pd.DataFrame(dic)
    
# optæl ord i sangtekst
print('generate number-of-words feature and normalize...', end='')
predict_df = word_count(predict_df, 'num_words', 'lyrics')
# normaliser antal ord i sangtekst
predict_df['num_words_nm'] = predict_df.num_words.apply(lambda val: (val-min_nw)/(max_nw-min_nw))
print('DONE')
    
# optæl gennemsnitlig ordlængde i sangtekst
print('generate avg-word-length feature and normalize...', end='')
#df_cp = df_cp[df_cp.lyrics.apply(type) == str] # lyrics MUST be type string
predict_df = sentence_avg_word_length(predict_df, 'avg_word_len', 'lyrics')
predict_df['avg_word_len_nm'] = predict_df.avg_word_len.apply(lambda val: (val-min_awl)/(max_awl-min_awl))
print('DONE')
    
# sentiments analyse
print('generate sentiments analyzis...', end='')
predict_df = analyze_sentiment(predict_df)
print('DONE')
    
# optæl ord-klasser i sangtekst
print('generate word-class counts...', end='')
predict_df = analyze_word_class(predict_df)
print('DONE')

input = predict_df[['num_words_nm','avg_word_len_nm','polarity', 'subjectivity', 'nouns', 'adverbs', 'verbs']]
input = np.asarray(input)
prediction = model.predict(input)

print()
print(f'model gætter på: {np.argmax(prediction)} som er {df_cp[df_cp.genre_code == np.argmax(prediction)].genre[0]}')

generate number-of-words feature and normalize...DONE
generate avg-word-length feature and normalize...DONE
generate sentiments analyzis...DONE
generate word-class counts...DONE

model gætter på: 5 som er Jazz
